In [49]:
import numpy as np
import pandas as pd

In [50]:
data=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/train.csv")
val=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/test.csv")
sub=pd.read_csv("/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv")

In [51]:
data=data.fillna(data.median())

/tmp/ipykernel_32/495656529.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data=data.fillna(data.median())


In [52]:
from sklearn.preprocessing import LabelEncoder

data['EJ'] = LabelEncoder().fit_transform(data['EJ'])
data=data.drop(['Id'], axis=1)

In [53]:
val['EJ'] = LabelEncoder().fit_transform(val['EJ'])
val=val.drop(['Id'], axis=1)

In [54]:
x=data.drop(["Class"], axis=1)
y=data["Class"]

In [55]:
#splitting data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [56]:
#making brier scorer
from sklearn.metrics import make_scorer, brier_score_loss

brier_scorer = make_scorer(brier_score_loss, greater_is_better=False)

In [57]:
brier_scorer

make_scorer(brier_score_loss, greater_is_better=False)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, cross_val_score

clf= GridSearchCV(xgb.XGBClassifier(objective='binary:logistic'),
{
    'n_estimators': [32,40,50,55,64,75,100],
    'learning_rate':[0.1,0.2,0.3,0.4,0.5],
    'max_depth':range(3,10),
    'colsample_bytree':[i/10.0 for i in range (1,10)],
    'gamma':[i/10.0 for i in range (1,10)]
}, cv=5, scoring=brier_scorer, return_train_score=False, n_jobs=-1)

clf.fit(x_train,y_train)
clf.cv_results_

In [ ]:
results=pd.DataFrame(clf.cv_results_)
results

In [ ]:
clf.best_params_

In [ ]:
results['mean_test_score'].max()


In [ ]:
import matplotlib.pyplot as plt

results.plot(x='param_n_estimators', y='mean_test_score', kind='line', marker='o')

# Display the plot
import matplotlib.pyplot as plt
plt.show()

In [ ]:
sorted_results = results.sort_values(by='std_test_score', ascending=True)
sorted_results

In [ ]:
sorted_results.to_csv('sorted.csv')

In [ ]:
predict = clf.predict(x_test)
print('Best AUC Score: {}'.format(clf.best_score_))
#print('Accuracy: {}'.format(brier_scorer(y_test, predict)))
#print(confusion_matrix(y_test,predict))

In [ ]:
pred = clf.predict_proba(val)
pred

In [ ]:
label= val.shape[0]

In [ ]:
pred=pred.astype(float)
pred = pd.DataFrame(pred)

#pred.resize(label,1)
pred.shape

In [ ]:
sub['class_0']=pred[0]
sub['class_1']=pred[1]

In [ ]:
sub


In [ ]:
sub.to_csv("submission.csv")
